In [1]:
%run init_notebook.py
from settings import DATA_DIR

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd

import os

from uuid import uuid4
import requests
import time

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By

In [3]:
ROOT_URL = "https://ec.europa.eu/commission/presscorner/advancedsearch/en?keywords=climate%" + \
            "20change&dotyp=4&parea=&pareaType=&datepickerbefore=12%20June%202017&datebefore=" + \
            "Mon%20Jun%2012%202017%2000:00:00%20GMT%2B0200%20(Central%20European%20Summer%20Time)&" + \
            "commissioner=&datepickerafter=10%20June%202016&dateafter=Fri%20Jun%2010%202016%2000:00:00%20GMT%2B0200%20" + \
            "(Central%20European%20Summer%20Time)&pagenumber="
url = ROOT_URL + "1"

ROOT_URL = "https://ec.europa.eu/commission/presscorner/advancedsearch/en?keywords=climate%20change&dotyp=4&parea=&pareaType=&datepickerbefore=31%20December%202023&datebefore=Sun%20Dec%2031%202023%2000:00:00%20GMT%2B0100%20(Central%20European%20Standard%20Time)&commissioner=&datepickerafter=10%20June%202013&dateafter=Mon%20Jun%2010%202013%2000:00:00%20GMT%2B0200%20(Central%20European%20Summer%20Time)"
url = ROOT_URL


In [4]:
print(url)

https://ec.europa.eu/commission/presscorner/advancedsearch/en?keywords=climate%20change&dotyp=4&parea=&pareaType=&datepickerbefore=31%20December%202023&datebefore=Sun%20Dec%2031%202023%2000:00:00%20GMT%2B0100%20(Central%20European%20Standard%20Time)&commissioner=&datepickerafter=10%20June%202013&dateafter=Mon%20Jun%2010%202013%2000:00:00%20GMT%2B0200%20(Central%20European%20Summer%20Time)


In [18]:
DRIVER_PAG = webdriver.Chrome(ChromeDriverManager().install())
DRIVER_SPEECH = webdriver.Chrome(ChromeDriverManager().install())

DRIVER_PAG.get(url)

In [19]:
speeches = {}

is_more_pages = True
while is_more_pages:
   
    # get all speech links from current pagination
    speech_links = [
        i for i in DRIVER_PAG.find_elements(By.TAG_NAME, 'a') 
        if i.get_attribute('class') == 'ecl-link ecl-list-item__link'
    ]
        
    # scrape speeches
    for idx, link in enumerate(speech_links):
        
        DRIVER_SPEECH.get(link.get_attribute('href'))
        
        text = DRIVER_SPEECH.find_elements(By.ID, 'inline-nav-1')[0].text
        date = DRIVER_SPEECH.find_element(By.NAME, 'Date').get_attribute('content')
        
        # save results to dict
        speeches[str(uuid4())] = {'text': text, 'date': date}  #'place': place}
            
    print('Scraped', len(speech_links), 'speeches')
    
    # advance to next pagintion
    page_pag = DRIVER_PAG.find_elements(By.CLASS_NAME, 'ecl-pager__wrapper') # 'ecl-pager__item ecl-pager__item-lNews') #'ecl-pager__link') #, 'div')
    assert len(page_pag) == 1, "more than one pagination found"
    
    # find new pages and current page in that list
    lst_page_pag_lis = page_pag[0].find_elements(By.TAG_NAME, 'li')
    mask = [i.get_attribute('class').split('ecl-pager__item ecl-pager__item')[-1] for i in lst_page_pag_lis]
    idx = [i[-7:] == 'current' for i in mask].index(True)
    
    # if more pages, advance
    if len(mask) > idx+2:
        try:
            button = lst_page_pag_lis[idx+1].find_element(By.TAG_NAME, 'a')
        except Exception as e:
            button = lst_page_pag_lis[idx+1]
            
#         print('Current page', button.text)''
        print('New page', button.text) #lst_page_pag_lis[idx+1].find_element(By.TAG_NAME, 'a').text)
        print('\n')

#         button = lst_page_pag_lis[idx+1].find_element(By.TAG_NAME, 'a')
        DRIVER_PAG.execute_script("arguments[0].click();", button)
        
        time.sleep(3)
    else:
        is_more_pages = False

    

Scraped 10 speeches
New page 2


Scraped 10 speeches
New page 3


Scraped 10 speeches
New page 4


Scraped 10 speeches
New page 5


Scraped 10 speeches
New page 6


Scraped 10 speeches
New page 7


Scraped 10 speeches
New page 8


Scraped 10 speeches
New page 9


Scraped 10 speeches
New page 10


Scraped 10 speeches
New page 11


Scraped 10 speeches
New page 12


Scraped 10 speeches
New page 13


Scraped 10 speeches
New page 14





KeyboardInterrupt



## output

In [23]:
pd.DataFrame(speeches).transpose() #.to_csv(os.path.join(DATA_DIR, 'ecb_speeches.csv'))

,text,date
2c1aee31-0a85-40ce-a823-8411b2004bf8,"""Check against delivery""\nSignore e Signori,\n...",2023-12-19
e2f90972-2214-4f3b-8c82-08f7adef82e0,"""Check against delivery""\nDear Lucia,\nDear An...",2023-12-17
216645b9-a138-4cb8-b173-105273a59f11,"""Check against delivery""\nMr. Bernard Arnault,...",2023-12-14
c157400a-438a-47e3-b41e-489546dc0878,"""Check against delivery""\nLadies and gentlemen...",2023-12-11
d5cf0cec-35e7-4d27-b68b-a055cad192cb,"""Check against delivery""\nDistinguished minist...",2023-12-09
...,...,...
1ac3c8e9-3046-4859-9e6d-207ede26934c,"""Check against delivery""\nYour Excellencies,\n...",2023-09-19
317c003a-64c7-47e0-a14d-e5ee0488899f,"""Check against delivery""\nSpeaking points\nExc...",2023-09-19
0a8319d1-654a-4c46-a453-b393caae926e,"""Check against delivery""\nThank you for your k...",2023-09-19
d6bac551-fc47-48ce-9294-64a5f8e7bd2f,"""Check against delivery""\nExcellencies, Minist...",2023-09-19
